In [1]:
import pandas as pd

# Transit

In [ ]:
transit_df = pd.read_excel(r'J:\Projects\Soundcast\network_summary_detailed.xlsx', sheetname='Transit Summaries')

In [63]:
# Reformat data
dict_result = {}
for field in ['board','time']:
    df = pd.DataFrame(transit_df[[tod+'_'+ field for tod in todlist]].stack())
    df.rename(columns={0:field}, inplace=True)
    df['tod'] = [i.split('_')[0] for i in df.index.get_level_values(1)]
    df['route_id'] = df.index.get_level_values(0)
    df.reset_index(inplace=True, drop=True)
    
    dict_result[field] = df


In [65]:
# Join route info

dict_result['board'].head()

,board,tod,route_id
0,82.318138,6to7,0001
1,128.641617,7to8,0001
2,97.873947,8to9,0001
3,94.789467,6to7,0002
4,104.986832,7to8,0002


# Traffic Counts

In [66]:
counts_df = pd.read_excel(r'J:\Projects\Soundcast\network_summary_detailed.xlsx', sheetname='Counts Output')

In [68]:
counts_df.head()

,,OBJECTID_1,Join_Count,TARGET_FID,OBJECTID,SR,RID,MP,ARM,Type_,Lanes,...,vol15to16,vol20to5,vol7to8,vol14to15,vol8to9,vol18to20,vol17to18,vol6to7,vol5to6,vol10to14
loop_INode,loop_JNode,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9282,10332,220,1,220,220,5,5,207.20,207.25,G,3,...,1273.474854,2736.361084,1777.173462,1240.454346,1430.036499,1726.923950,1361.388428,1416.392090,744.846375,4707.363281
10451,9277,188,1,188,188,5,5,207.20,207.25,G,3,...,1907.421753,2966.348633,1073.866577,1462.174438,926.935059,2798.736084,2252.753174,659.696289,344.664398,4534.764160
10732,13257,291,1,291,291,5,5,203.69,203.74,G,3,...,1448.459229,3784.408691,2443.908447,1525.198364,1874.788574,1946.188354,1490.086304,1989.342041,1031.138306,6036.317871
13260,10777,187,1,187,187,5,5,203.69,203.74,G,3,...,2429.952393,3802.711914,1196.982910,1808.074219,1014.486389,3631.598633,2863.095947,772.015198,411.879486,5469.338867


#  Network Summary

In [69]:
net_summary_df = pd.read_excel(r'J:\Projects\Soundcast\network_summary_detailed.xlsx', sheetname='Network Summary')

In [71]:
net_summary_df.stack()

9to10   TP_4k                     md
        arterial_vmt         1763401
        connectors_vmt      273600.6
        highway_vmt          2088964
        arterial_vht        63178.12
        connectors_vht      17905.68
        highway_vht            44905
        arterial_delay      14617.65
        connectors_delay      -2e-06
        highway_delay       7630.593
16to17  TP_4k                     pm
        arterial_vmt         2540470
        connectors_vmt        371387
        highway_vmt          2607158
        arterial_vht        97734.52
...
5to6    connectors_vht      6290.336
        highway_vht         19833.58
        arterial_delay      4182.004
        connectors_delay      -1e-06
        highway_delay       971.2118
10to14  TP_4k                     md
        arterial_vmt         8567331
        connectors_vmt       1443831
        highway_vmt          8687032
        arterial_vht        318087.5
        connectors_vht      94833.95
        highway_vht         183539